In [ ]:
import aqueduct as aq
from aqueduct.constants.enums import ArtifactType

In [ ]:
client = aq.Client(api_key="", aqueduct_address="")


aq.global_config({'engine':'databricks_resource', 'lazy':True})

In [ ]:
snowflake_warehouse = client.resource('snowflake_resource')
hotel_reviews = snowflake_warehouse.sql('SELECT * FROM large_hotel_reviews;')

In [ ]:
@aq.op(requirements=[])
def dummy(original_df, num_rows):
    from pyspark.sql.functions import monotonically_increasing_id
    from pyspark.sql.functions import rand
    import math
    
    original_row_count = original_df.count()
    num_partitions = int(math.ceil(num_rows / original_row_count))

    # Step 2: Repartition the DataFrame
    replicated_df = original_df.repartition(num_partitions)

    # Step 3: Persist the DataFrame
    replicated_df.persist()

    # Step 4: Duplicate the rows
    while replicated_df.count() < num_rows:
        replicated_df = replicated_df.union(replicated_df)

    print(replicated_df.count())

    return replicated_df

generated_df = dummy(hotel_reviews, 10000000000)

In [ ]:
snowflake_warehouse.save(generated_df, table_name="large_hotel_reviews", update_mode="replace")


client.publish_flow(
    "Creating_Large_Dataset",
    "repartition hotel_reviews to create big dataset",
    artifacts=[generated_df],
)